# Semantic Superposition with SD + KLMC2 via Prompt Rotation

<div>
<!--
<img src="https://images.squarespace-cdn.com/content/v1/6213c340453c3f502425776e/a432c21c-bb12-4f38-b5e2-1c12a3c403f6/Animated-Logo_1.gif" width="150"/>
<img src="https://github.com/dmarx/keyframed/raw/main/static/images/fancy.png" width="500"/>
-->
<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.translatorscafe.com%2Fstatic%2Fucvt%2Fimg%2Fcalc-three-phase-power-5.gif&f=1&nofb=1&ipt=20ef75a89759a599e5e1660cfaf8f8ac6e43554b5b35c7b9072c859b1d2c7f45&ipo=images" width="500"/>
</div>

*[image source](https://www.translatorscafe.com/unit-converter/nl-NL/calculator/three-phase-power/): Anatoly Zolotkov*


Notebook by [David Marx](https://twitter.com/DigThatData), based on [my fork](https://colab.research.google.com/github/dmarx/notebooks/blob/main/Stable_Diffusion_KLMC2_Animation.ipynb) of a [klmc2 notebook](https://colab.research.google.com/drive/1m8ovBpO2QilE2o4O-p2PONSwqGn4_x2G) by [Katherine Crowson](https://twitter.com/RiversHaveWings)

Sponsored by:

<div>
    <!--
<img src="https://images.squarespace-cdn.com/content/v1/6213c340453c3f502425776e/a432c21c-bb12-4f38-b5e2-1c12a3c403f6/Animated-Logo_1.gif" width="150"/>
-->
<img src="https://images.squarespace-cdn.com/content/v1/6213c340453c3f502425776e/4ef96ad9-e4c5-4c2b-8f21-15471ec16b9a/favicon.ico" />
</div>

[StabilityAI](https://twitter.com/stabilityai) 

---

## What is this?

This notebook dreams "superposition" animations by cyclically mixing prompts for [Stable Diffusion](https://stability.ai/blog/stable-diffusion-public-release) text-to-image generation. The morph that takes you from one frame to the next is accomplished using [KLMC2](https://arxiv.org/abs/1807.09382), which is sort of like exploring the latent by following the path of a heavy rolling ball.

Consider the gif above: each of your prompts is a vector, and the sine waves are each prompt's respective weight schedule. By cycling the strengths of the respective prompts relative to each other, we are performing a rotation in semantic space. The object we are rotating is the conditioning guidance, which acts like a kind of semantic magnet which pulls the current state of the image towards it to generate the next image, and KLMC2 is like a propulsion force on the image, smoothly pushing it forward through the semantic space. Depending on how strong and far apart things are and how fast they are changing, the magnet might have almost no effect on the trajectory of the image, or if the parameters are set carefully the image will hopefully "corkscrew" through semantic space. 

<img src="https://i1.wp.com/www.cosmic-core.org/wp-content/uploads/2018/10/Q-4-sine-waves-spirals3fresh-2.gif?resize=400%2C426&ssl=1" />

*[image source](https://www.cosmic-core.org/free/article-55-number-the-pentad-part-4-spirals-helices/): cosmic-core.org*

This corkscrew effect sometimes needs a warmup period before it hits its stride, similar to the "burn-in" that often accompanies MCMC methods. Use your early generations to determine if you like the seed but don't be surprised if the timing of changes in the image doesn't seem to map correctly to the weight schedules at first. The process seems to generally achieve stationarity -- i.e. find a sweet spot animation trajectory -- after about 300-500 frames.

<img src="https://raw.githubusercontent.com/markdregan/Bayesian-Modelling-in-Python/master/graphics/mcmc-animate.gif" />

*[image source](https://github.com/markdregan/Bayesian-Modelling-in-Python): Mark Regan, "Bayesian Modeling in Python"*

For advanced use, please reference [my KLMC2 Fork](https://colab.research.google.com/github/dmarx/notebooks/blob/main/Stable_Diffusion_KLMC2_Animation.ipynb), which this notebook is modified from. 

## Contact

Report bugs or feature ideas here: https://github.com/dmarx/notebooks/issues

In [ ]:
# @title Output Example 
from IPython.display import YouTubeVideo
YouTubeVideo('LklyRgTeqXI')

In [ ]:
#@title Check GPU
#!nvidia-smi

import pandas as pd
import subprocess

def gpu_info():
    outv = subprocess.run([
        'nvidia-smi',
            # these lines concatenate into a single query string
            '--query-gpu='
            'timestamp,'
            'name,'
            'utilization.gpu,'
            'utilization.memory,'
            'memory.used,'
            'memory.free,'
            ,
        '--format=csv'
        ],
        stdout=subprocess.PIPE).stdout.decode('utf-8')

    header, rec = outv.split('\n')[:-1]
    return pd.DataFrame({' '.join(k.strip().split('.')).capitalize():v for k,v in zip(header.split(','), rec.split(','))}, index=[0]).T

gpu_info()

In [ ]:
#@title Install Dependencies

# @markdown Your runtime will automatically restart after running this cell.
# @markdown You should only need to run this cell when setting up a new runtime. After future runtime restarts,
# @markdown you should be able to skip this cell.

import warnings

probably_using_colab = False
try:
    import google
    probably_using_colab = True
except ImportError:
    warnings.warn("Unable to import `google`, assuming this means we're using a local runtime")

# @markdown Not recommended for colab users. This notebook is currently configured to only make this
# @markdown option available for local install.
use_xformers = False

try:
    import keyframed
except ImportError:
    if probably_using_colab:
        !pip install ftfy einops braceexpand requests transformers clip open_clip_torch omegaconf pytorch-lightning kornia k-diffusion ninja omegaconf
        !pip install -U git+https://github.com/huggingface/huggingface_hub
        !pip install napm keyframed
    else:
        !pip install -r klmc2/requirements.txt
        if use_xformers:
            !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

    exit() # restarts the runtime

In [ ]:
# @markdown # Setup Workspace { display-mode: "form" }

###################
# Setup Workspace #
###################

import os
from pathlib import Path
import warnings

probably_using_colab = False
try:
    import google
    if Path('/content').exists():
        probably_using_colab = True
        print("looks like we're in colab")
    else:
        print("looks like we're not in colab")
except ImportError:
    warnings.warn("Unable to import `google`, assuming this means we're using a local runtime")


mount_gdrive = True # @param {type:'boolean'}

# defaults
outdir = Path('./frames')
if not os.environ.get('XDG_CACHE_HOME'):
    os.environ['XDG_CACHE_HOME'] = str(Path('~/.cache').expanduser())

if mount_gdrive and probably_using_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    Path('/content/drive/MyDrive/AI/models/.cache/').mkdir(parents=True, exist_ok=True) 
    os.environ['XDG_CACHE_HOME']='/content/drive/MyDrive/AI/models/.cache'
    outdir = Path('/content/drive/MyDrive/AI/klmc2/frames/')

# make sure the paths we need exist
outdir.mkdir(parents=True, exist_ok=True)
debug_dir = outdir.parent / 'debug_frames'
debug_dir.mkdir(parents=True, exist_ok=True)


os.environ['NAPM_PATH'] = str( Path(os.environ['XDG_CACHE_HOME']) / 'napm' )
Path(os.environ['NAPM_PATH']).mkdir(parents=True, exist_ok=True)


import napm

url = 'https://github.com/Stability-AI/stablediffusion'
napm.pseudoinstall_git_repo(url, add_install_dir_to_path=True)


##### Moved from model loading cell

if probably_using_colab:
    models_path = "/content/models" #@param {type:"string"}
else:
    models_path =  os.environ['XDG_CACHE_HOME']

if mount_gdrive and probably_using_colab:
    models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
    models_path = models_path_gdrive

if not Path(models_path).exists():
    Path(models_path).mkdir(parents=True, exist_ok=True)


In [ ]:
# @markdown # Imports and Definitions { display-mode: "form" }

###########
# imports #
###########

# importing napm puts the stable diffusion repo on the PATH, which is where `ldm` imports from
import napm
from ldm.util import instantiate_from_config

from base64 import b64encode
from collections import defaultdict
from concurrent import futures
import math
from pathlib import Path
import random
import re
import requests
from requests.exceptions import HTTPError
import sys
import time
from urllib.parse import urlparse
import warnings

import functorch
import huggingface_hub
from IPython.display import display, Video, HTML
import k_diffusion as K
from keyframed import Curve, ParameterGroup, SmoothCurve, CurveBase, SinusoidalCurve
import matplotlib.pyplot as plt
import numpy as np 
from omegaconf import OmegaConf
import PIL
from PIL import Image
import torch
from torch import nn
from tqdm.auto import tqdm, trange

from loguru import logger
import time
from natsort import natsorted


#########################
# Define useful globals #
#########################

cpu = torch.device("cpu")
device = torch.device("cuda")


##############################
# Define necessary functions #
##############################
    
import matplotlib.image
import numpy as np
    
def get_latest_frame(i=None, latest_frame_fpath=None):
    latest_frame = None
    if latest_frame_fpath is not None:
        latest_frame = latest_frame_fpath
    
    if (latest_frame is None) and (i is None):
        frames = Path('frames').glob("*.png")
        #logger.debug(f"frames: {len(frames)}")
        latest_frame = natsort.sort(frames)[-1]
        i = re.findall('out_([0-9]+).png', latest_frame.name)
    else:
        latest_frame = Path('frames') / f"out_{i:05}.png"
    logger.debug(f'-latest_frame: {latest_frame}')
    #return Image.open(latest_frame)
    img = matplotlib.image.imread(latest_frame)
    return np.flip(img, axis=0) # up/down

def plot_prompts(prompts=None, n=1000, settings=None, **kargs):
    if prompts is not None:
        for prompt in prompts:
            prompt.weight.plot(n=n, **kargs)

def plot_param(param, settings=None, prompts=None, n=1000, **kargs):
    settings.parameters[param].plot(n=n, **kargs)
            
# move imports up
import base64
from io import BytesIO
from functools import partial
            
@logger.catch
def write_debug_frame_at_(
    i=None,
    n=300, 
    prompts=None, 
    stuff_to_plot=['prompts'], 
    latest_frame_fpath=None,
    pil_image=None,
    settings=None,
):
    plotting_funcs = {
        'prompts': plot_prompts,
        'g': partial(plot_param, param='g'),
        'h': partial(plot_param, param='h'),
        'sigma': partial(plot_param, param='sigma'),
        'gamma': partial(plot_param, param='gamma'),
        'alpha': partial(plot_param, param='alpha'),
        'tau': partial(plot_param, param='tau'),
    }
    
    # i feel like this line of code justifies the silly variable name
    if not stuff_to_plot:
        return
    
    #stuff_to_plot = []
    
    test_im = pil_image
    if pil_image is None:
        test_im = get_latest_frame(i, latest_frame_fpath)

    fig = plt.figure()
    #axsRight = fig.subplots(3, 1, sharex=True)
    #ax = axsRight[0]
    ax_objs = fig.subplots(len(stuff_to_plot), 1, sharex=True)
    
    #width, height = test_im.size
    height, width = test_im.size
    fig.set_size_inches(height/fig.dpi, width/fig.dpi )
        
    buffer = BytesIO()
    for j, category in enumerate(stuff_to_plot):
        ax = ax_objs
        if len(stuff_to_plot) > 1:
            ax = ax_objs[j]
        plt.sca(ax)
        plt.tight_layout()
        plt.axis('off')
        
        plotting_funcs[category](prompts=prompts, settings=settings, n=n, zorder=1)
        plt.axvline(x=i)
         
        

    #plt.margins(0)
    fig.savefig(buffer, transparent=True)        
    plt.close()

    buffer.seek(0)
    plot_pil = Image.open(buffer)
        #buffer.close() # throws error here

    #debug_im_path = Path('debug_frames') / f"{category}_out_{i:05}.png"
    #debug_im_path = Path('debug_frames') / f"debug_out_{i:05}.png"
    debug_im_path = debug_dir / f"debug_out_{i:05}.png"
    test_im = test_im.convert('RGBA')
    test_im.paste(plot_pil, (0,0), plot_pil)
    test_im.save(debug_im_path)
    #display(test_im) # maybe?
    buffer.close() # I guess?
    
    return test_im, plot_pil

##############################

class Prompt:
    def __init__(
        self,
        text,
        weight_schedule,
    ):
        c = sd_model.get_learned_conditioning([text])
        self.text=text
        self.encoded=c
        if not isinstance(weight_schedule, CurveBase):
            weight_schedule = SmoothCurve(weight_schedule)
        self.weight = weight_schedule


def handle_chigozienri_curve_format(value_string):
    if value_string.startswith('(') and value_string.endswith(')'):
        value_string = value_string[1:-1]
    return value_string

def parse_curve_string(txt, f=float):
    schedule = {}
    for tokens in txt.split(','):
        k,v = tokens.split(':')
        v = handle_chigozienri_curve_format(v)
        schedule[int(k)] = f(v)
    return schedule

def parse_curvable_string(param, is_int=False):
    if isinstance(param, dict):
        return param
    f = float
    if is_int:
        f = int
    try:
        return f(param)
    except ValueError:
        return parse_curve_string(txt=param, f=f)

##################

def show_video(video_path, video_width=512):
    return display(Video(video_path, width=video_width))

if probably_using_colab:
    def show_video(video_path, video_width=512):
        video_file = open(video_path, "r+b").read()
        video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
        return display(HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>"""))

##################

class NormalizingCFGDenoiser(nn.Module):
    def __init__(self, model, g):
        super().__init__()
        self.inner_model = model
        self.g = g
        self.eps_norms = defaultdict(lambda: (0, 0))

    def mean_sq(self, x):
        return x.pow(2).flatten(1).mean(1)

    @torch.no_grad()
    def update_eps_norm(self, eps, sigma):
        sigma = sigma[0].item()
        eps_norm = self.mean_sq(eps).mean()
        eps_norm_avg, count = self.eps_norms[sigma]
        eps_norm_avg = eps_norm_avg * count / (count + 1) + eps_norm / (count + 1)
        self.eps_norms[sigma] = (eps_norm_avg, count + 1)
        return eps_norm_avg

    def forward(self, x, sigma, uncond, cond, g):
        x_in = torch.cat([x] * 2)
        sigma_in = torch.cat([sigma] * 2)
        cond_in = torch.cat([uncond, cond])

        denoised = self.inner_model(x_in, sigma_in, cond=cond_in)
        eps = K.sampling.to_d(x_in, sigma_in, denoised)
        eps_uc, eps_c = eps.chunk(2)
        eps_norm = self.update_eps_norm(eps, sigma).sqrt()
        c = eps_c - eps_uc
        cond_scale = g * eps_norm / self.mean_sq(c).sqrt()
        eps_final = eps_uc + c * K.utils.append_dims(cond_scale, x.ndim)
        return x - eps_final * K.utils.append_dims(sigma, eps.ndim)

#########################

def write_klmc2_state(**state):
    st = time.time()
    obj = {}
    for k,v in state.items():
        try:
            v = v.clone().detach().cpu()
        except AttributeError:
            # if it doesn't have a detach method, we don't need to worry about any preprocessing
            pass
        obj[k] = v

    checkpoint_fpath = Path(outdir) / f"klmc2_state_{state.get('i',0):05}.ckpt"
    with open(checkpoint_fpath, 'wb') as f:
        torch.save(obj, f=f)
    et = time.time()
    #logger.debug(f"checkpointing: {et-st}")
    # to do: move to callback? thread executor, anyway

def read_klmc2_state(root=outdir, latest_frame=-1):
    state = {}
    checkpoints = [str(p) for p in Path(root).glob("*.ckpt")]
    if not checkpoints:
        return None
    checkpoints = natsorted(checkpoints)
    if latest_frame < 0:
        ckpt_fpath = checkpoints[-1]
    else:
        for fname in checkpoints:
            frame_id = re.findall(r'([0-9]+).ckpt', fname)[0]
            if int(frame_id) <= latest_frame:
                ckpt_fpath = fname
            else:
                break
    logger.debug(ckpt_fpath)
    with open(ckpt_fpath,'rb') as f:
        state = torch.load(f=f,map_location='cuda')
    return state

def load_init_image(init_image, height, width):
    if not Path(init_image).exists():
        raise FileNotFoundError(f"Unable to locate init image from path: {init_image}")
    
    
    from PIL import Image
    import numpy as np

    init_im_pil = Image.open(init_image)

    #x_pil = init_im_pil.resize([512,512])
    x_pil = init_im_pil.resize([height,width])
    x_np = np.array(x_pil.convert('RGB')).astype(np.float16) / 255.0
    x = x_np[None].transpose(0, 3, 1, 2)
    x = 2.*x - 1.
    x = torch.from_numpy(x).to('cuda')
    return x

def save_image_fn(image, name, i, n, prompts=None, settings=None, stuff_to_plot=['prompts']):
    pil_image = K.utils.to_pil_image(image)
    if i % 10 == 0 or i == n - 1:
        print(f'\nIteration {i}/{n}:')
        display(pil_image)
    if i == n - 1:
        print('\nDone!')
    pil_image.save(name)
    if stuff_to_plot:
        #logger.debug(stuff_to_plot)
        #write_debug_frame_at_(i, prompts=prompts)
        debug_frame, debug_plot = write_debug_frame_at_(i=i,n=n, prompts=prompts, settings=settings, stuff_to_plot=stuff_to_plot, pil_image=pil_image)
        if i % 10 == 0 or i == n - 1:
            #display(debug_frame)
            display(debug_plot)

###############################

@torch.no_grad()
def sample_mcmc_klmc2(
    sd_model, 
    init_image,
    height:int,
    width:int,
    n:int, 
    hvp_method:str='reverse', 
    prompts:list=None,
    settings:ParameterGroup=None,
    resume:bool = False,
    resume_from:int=-1,
    img_init_steps:int=None,
    stuff_to_plot:list=None,
    checkpoint_every:int=10,
):

    if stuff_to_plot is None:
        stuff_to_plot = ['prompts','h']
    
    torch.cuda.empty_cache()

    wrappers = {'eps': K.external.CompVisDenoiser, 'v': K.external.CompVisVDenoiser}
    g = settings[0]['g']

    model_wrap = wrappers[sd_model.parameterization](sd_model)
    model_wrap_cfg = NormalizingCFGDenoiser(model_wrap, g)
    sigma_min, sigma_max = model_wrap.sigmas[0].item(), model_wrap.sigmas[-1].item()
    model = model_wrap_cfg

    uc = sd_model.get_learned_conditioning([''])
    extra_args = {
        'uncond': uc, 
        #'cond': prompts[0].encoded,
        'g': settings[0]['g']
    }

    sigma = settings[0]['sigma']

    with torch.cuda.amp.autocast(), futures.ThreadPoolExecutor() as ex:
        def callback(info):
            i = info['i']
            rgb = sd_model.decode_first_stage(info['denoised'] )
            ex.submit(save_image_fn, image=rgb, name=(outdir / f'out_{i:05}.png'), i=i, n=n, prompts=prompts, settings=settings, stuff_to_plot=stuff_to_plot)

        # Initialize the chain
        print('Initializing the chain...')

        # to do: if resuming, generating this init image is unnecessary
        x = None
        if init_image:
            print("loading init image")
            x = load_init_image(init_image, height, width)
            # convert RGB to latent
            x = sd_model.get_first_stage_encoding(sd_model.encode_first_stage(x))
            print("init image loaded.")

        print('Actually doing the sampling...')

        i_resume=0
        v = None
        if resume:
            state = read_klmc2_state(latest_frame=resume_from)
            if state:
                x, v, i_resume = state['x'], state['v'], state['i']
                # to do: resumption of settings
                settings_i = state['settings_i']
                i = i_resume
                settings[i]['h'] = settings_i['h']
                settings[i]['gamma'] = settings_i['gamma']
                settings[i]['alpha'] = settings_i['alpha']
                settings[i]['tau'] = settings_i['tau']
                settings[i]['g'] = settings_i['g']
                settings[i]['sigma'] = settings_i['sigma']
                settings[i]['steps'] = settings_i['steps']
        
        # to do: use multicond for init image
        # we want this test after resumption if resuming
        if x is None:
            print("No init image provided, generating a random init image")
            extra_args['cond'] = prompts[0].encoded
            h=height//8
            w=width//8
            x = torch.randn([1, 4, h, w], device=device) * sigma_max
            sigmas_pre = K.sampling.get_sigmas_karras(img_init_steps, sigma, sigma_max, device=x.device)[:-1]
            x = K.sampling.sample_dpmpp_sde(model_wrap_cfg, x, sigmas_pre, extra_args=extra_args)

        # if not resuming, randomly initialize momentum
        # this needs to be *after* generating X if we're going to...
        if v is None:
            v = torch.randn_like(x) * sigma

        # main sampling loop
        for i in trange(n):
            # fast-forward loop to resumption index
            if resume and i < i_resume:
                continue
            # if resume and (i == i_resume):
            #     # should these values be written into settings[i]?
            #     h = settings_i['h']
            #     gamma = settings_i['gamma']
            #     alpha = settings_i['alpha']
            #     tau = settings_i['tau']
            #     g = settings_i['g']
            #     sigma = settings_i['sigma']
            #     steps = settings_i['steps']
            # else:
            h = settings[i]['h']
            gamma = settings[i]['gamma']
            alpha = settings[i]['alpha']
            tau = settings[i]['tau']
            g = settings[i]['g']
            sigma = settings[i]['sigma']
            steps = settings[i]['steps']

            h = torch.tensor(h, device=x.device)
            gamma = torch.tensor(gamma, device=x.device)
            alpha = torch.tensor(alpha, device=x.device)
            tau = torch.tensor(tau, device=x.device)
            sigma = torch.tensor(sigma, device=x.device)
            steps = int(steps)
            
            sigmas = K.sampling.get_sigmas_karras(steps, sigma_min, sigma.item(), device=x.device)[:-1]

            x, v, grad = klmc2_step(
                model,
                prompts,
                x,
                v,
                h,
                gamma,
                alpha,
                tau,
                g,
                sigma,
                sigmas,
                steps,
                hvp_method,
                i,
                callback,
                extra_args,
            )

            save_checkpoint = (i % checkpoint_every) == 0
            if save_checkpoint:
                settings_i = settings[i]
                ex.submit(write_klmc2_state, v=v, x=x, i=i, settings_i=settings_i)
                logger.debug(settings[i])


def hvp_fn_forward_functorch(model, x, sigma, v, alpha, extra_args):
    def grad_fn(x, sigma):
        s_in = x.new_ones([x.shape[0]])
        denoised = model(x, sigma * s_in, **extra_args)
        return (x - denoised) + alpha * x
    jvp_fn = lambda v: functorch.jvp(grad_fn, (x, sigma), (v, torch.zeros_like(sigma)))
    grad, jvp_out = functorch.vmap(jvp_fn)(v)
    return grad[0], jvp_out

def hvp_fn_reverse(model, x, sigma, v, alpha, extra_args):
    def grad_fn(x, sigma):
        s_in = x.new_ones([x.shape[0]])
        denoised = model(x, sigma * s_in, **extra_args)
        return (x - denoised) + alpha * x
    vjps = []
    with torch.enable_grad():
        x_ = x.clone().requires_grad_()
        grad = grad_fn(x_, sigma)
        for k, item in enumerate(v):
            vjp_out = torch.autograd.grad(grad, x_, item, retain_graph=k < len(v) - 1)[0]
            vjps.append(vjp_out)
    return grad, torch.stack(vjps)

def hvp_fn_zero(model, x, sigma, v, alpha, extra_args):
    def grad_fn(x, sigma):
        s_in = x.new_ones([x.shape[0]])
        denoised = model(x, sigma * s_in, **extra_args)
        return (x - denoised) + alpha * x
    return grad_fn(x, sigma), torch.zeros_like(v)

def hvp_fn_fake(model, x, sigma, v, alpha, extra_args):
    def grad_fn(x, sigma):
        s_in = x.new_ones([x.shape[0]])
        denoised = model(x, sigma * s_in, **extra_args)
        return (x - denoised) + alpha * x
    return grad_fn(x, sigma), (1 + alpha) * v


def multicond_hvp(model, x, sigma, v, alpha, extra_args, prompts, hvp_fn, i):

    # loop over prompts and aggregate gradients for multicond
    grad = torch.zeros_like(x)
    h2_v = torch.zeros_like(x)
    h2_noise_v2 = torch.zeros_like(x)
    h2_noise_x2 = torch.zeros_like(x)
    wt_norm = 0
    for prompt in prompts:
        wt = prompt.weight[i]
        if wt == 0:
            continue
        wt_norm += wt
        wt = torch.tensor(wt, device=x.device)
        extra_args['cond'] = prompt.encoded

        # Estimate gradient and hessian
        grad_, (h2_v_, h2_noise_v2_, h2_noise_x2_) = hvp_fn(
            model=model,
            x=x, 
            sigma=sigma, 
            v=v,
            alpha=alpha,
            extra_args=extra_args,
        )

        grad = grad + grad_ * wt 
        h2_v = h2_v + h2_v_ * wt
        h2_noise_v2 = h2_noise_v2 + h2_noise_v2_ * wt
        h2_noise_x2 = h2_noise_x2 + h2_noise_x2_ * wt

    # Normalize gradient to magnitude it'd have if just single prompt w/ wt=1.
    # simplifies multicond w/o deep frying image or adding hyperparams
    grad = grad / wt_norm 
    h2_v = h2_v / wt_norm
    h2_noise_v2 = h2_noise_v2 / wt_norm
    h2_noise_x2 = h2_noise_x2 / wt_norm

    return grad, h2_v, h2_noise_v2, h2_noise_v2, h2_noise_x2




def klmc2_step(
            model,
            prompts,
            x,
            v,
            h,
            gamma,
            alpha,
            tau,
            g,
            sigma,
            sigmas,
            steps,
            hvp_method,
            i,
            callback,
            extra_args,
        ):

    #s_in = x.new_ones([x.shape[0]])

    # Model helper functions

    hvp_fns = {'forward-functorch': hvp_fn_forward_functorch,
                'reverse': hvp_fn_reverse,
                'zero': hvp_fn_zero,
                'fake': hvp_fn_fake}

    hvp_fn = hvp_fns[hvp_method]

    # KLMC2 helper functions
    def psi_0(gamma, t):
        return torch.exp(-gamma * t)

    def psi_1(gamma, t):
        return -torch.expm1(-gamma * t) / gamma

    def psi_2(gamma, t):
        return (torch.expm1(-gamma * t) + gamma * t) / gamma ** 2

    def phi_2(gamma, t_):
        t = t_.double()
        out = (torch.exp(-gamma * t) * (torch.expm1(gamma * t) - gamma * t)) / gamma ** 2
        return out.to(t_)

    def phi_3(gamma, t_):
        t = t_.double()
        out = (torch.exp(-gamma * t) * (2 + gamma * t + torch.exp(gamma * t) * (gamma * t - 2))) / gamma ** 3
        return out.to(t_)


    # Compute model outputs and sample noise
    x_trapz = torch.linspace(0, h, 1001, device=x.device)
    y_trapz = [fun(gamma, x_trapz) for fun in (psi_0, psi_1, phi_2, phi_3)]
    noise_cov = torch.tensor([[torch.trapz(y_trapz[i] * y_trapz[j], x=x_trapz) for j in range(4)] for i in range(4)], device=x.device)
    noise_v, noise_x, noise_v2, noise_x2 = torch.distributions.MultivariateNormal(x.new_zeros([4]), noise_cov).sample(x.shape).unbind(-1)

    extra_args['g']=g

    # compute derivatives, multicond wrapper loops over prompts and averages derivatives
    grad, h2_v, h2_noise_v2, h2_noise_v2, h2_noise_x2 = multicond_hvp(
        model=model, 
        x=x, 
        sigma=sigma, 
        v=torch.stack([v, noise_v2, noise_x2]), # need a "dummy" v for init image generation
        alpha=alpha, 
        extra_args=extra_args, 
        prompts=prompts, 
        hvp_fn=hvp_fn,
        i=i,
    )

    # DPM-Solver++(2M) refinement steps
    x_refine = x
    use_dpm = True
    old_denoised = None
    for j in range(len(sigmas) - 1):
        if j == 0:
            denoised = x_refine - grad
        else:
            s_in = x.new_ones([x.shape[0]])
            denoised = model(x_refine, sigmas[j] * s_in, **extra_args)
        dt_ode = sigmas[j + 1] - sigmas[j]
        if not use_dpm or old_denoised is None or sigmas[j + 1] == 0:
            eps = K.sampling.to_d(x_refine, sigmas[j], denoised)
            x_refine = x_refine + eps * dt_ode
        else:
            h_ode = sigmas[j].log() - sigmas[j + 1].log()
            h_last = sigmas[j - 1].log() - sigmas[j].log()
            fac = h_ode / (2 * h_last)
            denoised_d = (1 + fac) * denoised - fac * old_denoised
            eps = K.sampling.to_d(x_refine, sigmas[j], denoised_d)
            x_refine = x_refine + eps * dt_ode
        old_denoised = denoised
    if callback is not None:
        callback({'i': i, 'denoised': x_refine})

    # Update the chain
    noise_std = (2 * gamma * tau * sigma ** 2).sqrt()
    v_next = 0 + psi_0(gamma, h) * v - psi_1(gamma, h) * grad - phi_2(gamma, h) * h2_v + noise_std * (noise_v - h2_noise_v2)
    x_next = x + psi_1(gamma, h) * v - psi_2(gamma, h) * grad - phi_3(gamma, h) * h2_v + noise_std * (noise_x - h2_noise_x2)
    v, x = v_next, x_next

    return x, v, grad 

In [ ]:
#@markdown **Select and Load Model**

## TO DO:
## - if local, try to load model from ~/.cache/huggingface/diffusers

# modified from:
#   https://github.com/deforum/stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb

import napm
from ldm.util import instantiate_from_config


model_config = "v1-inference.yaml" #@param ["custom","v1-inference.yaml"]
model_checkpoint =  "sd-v1-4.ckpt" #@param ["custom","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","waifu-diffusion-v1-3.ckpt"]
if model_checkpoint == "waifu-diffusion-v1-3.ckpt":
    model_checkpoint = "model-epoch05-float16.ckpt"
custom_config_path = "" #@param {type:"string"}
custom_checkpoint_path = "" #@param {type:"string"}

half_precision = True # check
check_sha256 = False #@param {type:"boolean"}

model_map = {
    "sd-v1-4-full-ema.ckpt": {
        'sha256': '14749efc0ae8ef0329391ad4436feb781b402f4fece4883c7ad8d10556d8a36a',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-2-original/blob/main/sd-v1-4-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-4.ckpt": {
        'sha256': 'fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt',
        'requires_login': True,
        },
    "sd-v1-3-full-ema.ckpt": {
        'sha256': '54632c6e8a36eecae65e36cb0595fab314e1a1545a65209f24fde221a8d4b2ca',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-3-original/blob/main/sd-v1-3-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-3.ckpt": {
        'sha256': '2cff93af4dcc07c3e03110205988ff98481e86539c51a8098d4f2236e41f7f2f',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-3-original/resolve/main/sd-v1-3.ckpt',
        'requires_login': True,
        },
    "sd-v1-2-full-ema.ckpt": {
        'sha256': 'bc5086a904d7b9d13d2a7bccf38f089824755be7261c7399d92e555e1e9ac69a',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-2-original/blob/main/sd-v1-2-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-2.ckpt": {
        'sha256': '3b87d30facd5bafca1cbed71cfb86648aad75d1c264663c0cc78c7aea8daec0d',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-2-original/resolve/main/sd-v1-2.ckpt',
        'requires_login': True,
        },
    "sd-v1-1-full-ema.ckpt": {
        'sha256': 'efdeb5dc418a025d9a8cc0a8617e106c69044bc2925abecc8a254b2910d69829',
        'url':'https://huggingface.co/CompVis/stable-diffusion-v-1-1-original/resolve/main/sd-v1-1-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-1.ckpt": {
        'sha256': '86cd1d3ccb044d7ba8db743d717c9bac603c4043508ad2571383f954390f3cea',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-1-original/resolve/main/sd-v1-1.ckpt',
        'requires_login': True,
        },
    "robo-diffusion-v1.ckpt": {
        'sha256': '244dbe0dcb55c761bde9c2ac0e9b46cc9705ebfe5f1f3a7cc46251573ea14e16',
        'url': 'https://huggingface.co/nousr/robo-diffusion/resolve/main/models/robo-diffusion-v1.ckpt',
        'requires_login': False,
        },
    "model-epoch05-float16.ckpt": {
        'sha256': '26cf2a2e30095926bb9fd9de0c83f47adc0b442dbfdc3d667d43778e8b70bece',
        'url': 'https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/model-epoch05-float16.ckpt',
        'requires_login': False,
        },
}

# config path
ckpt_config_path = custom_config_path if model_config == "custom" else os.path.join(models_path, model_config)
if os.path.exists(ckpt_config_path):
    print(f"{ckpt_config_path} exists")
else:
    #ckpt_config_path = "./stable-diffusion/configs/stable-diffusion/v1-inference.yaml"
    ckpt_config_path = "./v1-inference.yaml"
    if not Path(ckpt_config_path).exists():
        !wget https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
    
print(f"Using config: {ckpt_config_path}")

# checkpoint path or download
ckpt_path = custom_checkpoint_path if model_checkpoint == "custom" else os.path.join(models_path, model_checkpoint)
ckpt_valid = True
if os.path.exists(ckpt_path):
    print(f"{ckpt_path} exists")
elif 'url' in model_map[model_checkpoint]:
    url = model_map[model_checkpoint]['url']

    # CLI dialogue to authenticate download
    if model_map[model_checkpoint]['requires_login']:
        print("This model requires an authentication token")
        print("Please ensure you have accepted its terms of service before continuing.")

        username = input("What is your huggingface username?:")
        token = input("What is your huggingface token?:")

        _, path = url.split("https://")

        url = f"https://{username}:{token}@{path}"

    # contact server for model
    print(f"Attempting to download {model_checkpoint}...this may take a while")
    ckpt_request = requests.get(url)
    request_status = ckpt_request.status_code

    # inform user of errors
    if request_status == 403:
      raise ConnectionRefusedError("You have not accepted the license for this model.")
    elif request_status == 404:
      raise ConnectionError("Could not make contact with server")
    elif request_status != 200:
      raise ConnectionError(f"Some other error has ocurred - response code: {request_status}")

    # write to model path
    with open(os.path.join(models_path, model_checkpoint), 'wb') as model_file:
        model_file.write(ckpt_request.content)
else:
    print(f"Please download model checkpoint and place in {os.path.join(models_path, model_checkpoint)}")
    ckpt_valid = False

if check_sha256 and model_checkpoint != "custom" and ckpt_valid:
    import hashlib
    print("\n...checking sha256")
    with open(ckpt_path, "rb") as f:
        bytes = f.read() 
        hash = hashlib.sha256(bytes).hexdigest()
        del bytes
    if model_map[model_checkpoint]["sha256"] == hash:
        print("hash is correct\n")
    else:
        print("hash in not correct\n")
        ckpt_valid = False

if ckpt_valid:
    print(f"Using ckpt: {ckpt_path}")

def load_model_from_config(config, ckpt, verbose=False, device='cuda', half_precision=True):
    map_location = "cuda" #@param ["cpu", "cuda"]
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location=map_location)
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    if half_precision:
        model = model.half().to(device)
    else:
        model = model.to(device)
    model.eval()
    return model

if ckpt_valid:
    local_config = OmegaConf.load(f"{ckpt_config_path}")
    model = load_model_from_config(local_config, f"{ckpt_path}", half_precision=half_precision)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)

    # Disable checkpointing as it is not compatible with the method
    for module in model.modules():
        if hasattr(module, 'checkpoint'):
            module.checkpoint = False
        if hasattr(module, 'use_checkpoint'):
            module.use_checkpoint = False

    sd_model=model

####################################################################

use_new_vae = True #@param {type:"boolean"}

if use_new_vae:

    # from kat's notebook again

    def download_from_huggingface(repo, filename):
        while True:
            try:
                return huggingface_hub.hf_hub_download(repo, filename)
            except HTTPError as e:
                if e.response.status_code == 401:
                    # Need to log into huggingface api
                    huggingface_hub.interpreter_login()
                    continue
                elif e.response.status_code == 403:
                    # Need to do the click through license thing
                    print(f'Go here and agree to the click through license on your account: https://huggingface.co/{repo}')
                    input('Hit enter when ready:')
                    continue
                else:
                    raise e

    vae_840k_model_path = download_from_huggingface("stabilityai/sd-vae-ft-mse-original", "vae-ft-mse-840000-ema-pruned.ckpt")

    def load_model_from_config_kc(config, ckpt):
        print(f"Loading model from {ckpt}")
        pl_sd = torch.load(ckpt, map_location="cpu")
        sd = pl_sd["state_dict"]
        config = OmegaConf.load(config)

        try:
            config['model']['params']['lossconfig']['target'] = "torch.nn.Identity"
            print('Patched VAE config.')
        except KeyError:
            pass

        model = instantiate_from_config(config.model)
        m, u = model.load_state_dict(sd, strict=False)
        model = model.to(cpu).eval().requires_grad_(False)
        return model

    vaemodel_yaml_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/models/first_stage_models/kl-f8/config.yaml"
    vaemodel_yaml_fname = 'config_vae_kl-f8.yaml'
    vaemodel_yaml_fname_git = "latent-diffusion/models/first_stage_models/kl-f8/config.yaml"
    if Path(vaemodel_yaml_fname_git).exists():
        vae_model = load_model_from_config_kc(vaemodel_yaml_fname_git, vae_840k_model_path).half().to(device)
    else:
        if not Path(vaemodel_yaml_fname).exists():
            !wget {vaemodel_yaml_url} -O {vaemodel_yaml_fname}
        vae_model = load_model_from_config_kc(vaemodel_yaml_fname, vae_840k_model_path).half().to(device)

    del sd_model.first_stage_model
    sd_model.first_stage_model = vae_model

In [ ]:
# @title Basic Settings

# @markdown Separate prompts with "|". 
# @markdown If an init image is not provided, the first prompt will be used to generate one.
prompt="new york style thin crust pizza | chicago style deep dish pizza | sheppherds pie " # @param {type:'string'}


# @markdown The number of frames to sample:
n = 1000 # @param {type:"integer"}

# @markdown height and width must be multiples of 8 (e.g. 256, 512, 768, 1024)
height = 512 # @param {type:"integer"}

width = 512 # @param {type:"integer"}


# @markdown If seed is negative, a random seed will be used
seed = -1 # @param {type:"number"}

init_image = "" # @param {type:'string'}



In [ ]:

# @title Advanced Settings

# @markdown ## Prompt Settings

# @markdown default settings have prompt weight oscillating between 0.8 and 1.2,
# @markdown rising from its lowest to highest weight over a 100 frame span.
prompt_weight_mean = 1 # @param {type:'number'}
prompt_weight_amplitude = 0.2 # @param {type:'number'}
prompt_weight_wavelength = 200 # @param {type:'number'}

################################

# @markdown ## KLMC2 Settings

# @markdown The strength of the conditioning on the prompt (0-1):
g="0.08"  # @param {type:"string"}

# @markdown The noise level to sample at (0.5-3)
sigma = "2.5" # @param {type:"string"}

# @markdown Step size (0-1):
h = "0.08" # @param {type:"string"}

# @markdown Friction (2 is critically damped, lower -> smoother animation):
gamma = "1.1" # @param {type:"string"}

# @markdown Quadratic penalty ("weight decay") strength:
alpha = "0.005" # @param {type:"string"}

# @markdown Temperature (adjustment to the amount of noise added per step):
tau = "1.0" # @param {type:"string"}

# @markdown Denoising refinement steps:
refinement_steps = "6" # @param {type:"string"}

# @markdown If an init image is not provided, this is how many steps will be used when generating an initial state:
img_init_steps = 15 # @param {type:"number"}

# @markdown The HVP method:
# @markdown <br><small>`forward-functorch` and `reverse` provide real second derivatives. Compatibility, speed, and memory usage vary by model and xformers configuration.
# @markdown `fake` is very fast and low memory but inaccurate. `zero` (fallback to first order KLMC) is not recommended.</small>
hvp_method = 'fake' # @param ["fake", "reverse", "forward-functorch", "zero"]

checkpoint_every = 10 # @param {type:"number"}

###########################


assert (height % 8) == 0
assert (width % 8) == 0


#############################################

#############################################


plot_prompt_weight_curves = True # @#param {type: 'boolean'}
plot_settings_weight_curves = False # @#param {type: 'boolean'}


#####################################

prompt_texts = prompt.split('|')
m = len(prompt_texts)

prompts = [
    Prompt(txt,
           prompt_weight_mean + SinusoidalCurve(
               amplitude=prompt_weight_amplitude,
               wavelength=prompt_weight_wavelength,
               phase=2*math.pi * (m-j-1)/m
           ))
    for j, txt in enumerate(prompt_texts)
]

#############################################

# Build Settings object

g = parse_curvable_string(g)
sigma = parse_curvable_string(sigma)
h = parse_curvable_string(h)
gamma = parse_curvable_string(gamma)
alpha = parse_curvable_string(alpha)
tau = parse_curvable_string(tau)
steps = parse_curvable_string(refinement_steps)


curved_settings = ParameterGroup({
    'g':SmoothCurve(g),
    'sigma':SmoothCurve(sigma),
    'h':SmoothCurve(h),
    'gamma':SmoothCurve(gamma),
    'alpha':SmoothCurve(alpha),
    'tau':SmoothCurve(tau),
    'steps':SmoothCurve(steps),
})


if plot_prompt_weight_curves:
    for prompt_ in prompts:
        prompt_.weight.plot(n=n)
    plt.title("prompt weight schedules")
    plt.show()


if plot_settings_weight_curves:
    for name, curve in curved_settings.parameters.items():
        curve.plot(n=n)
        plt.title(name)
        plt.show()

In [ ]:
# load settings from disk

load_settings_from_disk = False # @param {type:'boolean'}
load_prompts_from_disk = False  # @param {type:'boolean'}


from numbers import Number

if load_settings_from_disk:
    with open(outdir / 'settings.yaml', 'r') as f:
        curved_settings = keyframed.serialization.from_yaml(f.read())

#curved_settings.plot()

###########################

if load_prompts_from_disk:
    with open(outdir / 'prompts.yaml', 'r') as f:
        prompts_cfg = OmegaConf.load(f)

    prompts = []
    for p in prompts_cfg.prompts:
        if isinstance(p.weight, Number):
            weight_curve = SmoothCurve(p.weight)
        else:
            weight_curve = keyframed.serialization.from_yaml(OmegaConf.to_yaml(p.weight))
        P = Prompt(text=p.prompt, weight_schedule=weight_curve)
        prompts.append(P)
        #P.weight.plot()

In [ ]:
#@title Generate Animation Frames

resume = True # @param {type:'boolean'}
archive_old_work = False # @param {type:'boolean'}

# -1 = most recent frame
resume_from = -1 # @param {type:'number'}

# @markdown optional debugging plot
plot_prompt_weights = False # @param {type:'boolean'}

plot_h = False # @#param {type:'boolean'}
plot_g = False # @#param {type:'boolean'}
plot_sigma = False # @#param {type:'boolean'}
plot_gamma = False # @#param {type:'boolean'}
plot_alpha = False # @#param {type:'boolean'}
plot_tau = False # @#param {type:'boolean'}

################

_seed = seed
if seed < 0:    
    _seed = random.randrange(0, 4294967295)
print(f"using seed: {_seed}")
torch.manual_seed(_seed)

stuff_to_plot = []
if plot_prompt_weights:
    stuff_to_plot.append('prompts')
if plot_h:
    stuff_to_plot.append('h')
if plot_g:
    stuff_to_plot.append('g')
if plot_sigma:
    stuff_to_plot.append('sigma')
if plot_gamma:
    stuff_to_plot.append('gamma')
if plot_alpha:
    stuff_to_plot.append('alpha')
if plot_tau:
    stuff_to_plot.append('tau')

if not resume:
    if archive_old_work:
        archive_dir = outdir.parent / 'archive' / str(int(time.time()))
        archive_dir.mkdir(parents=True, exist_ok=True)
        print(f"Archiving contents of /frames, moving to: {archive_dir}")
    else:
        print("Old contents of /frames being deleted. This can be prevented in the future by setting either 'resume' or 'archive_old_work' to True.")
    for p in outdir.glob(f'*'):
        if archive_old_work:
            target = archive_dir / p.name
            p.rename(target)
        else:
            p.unlink()
    for p in Path('debug_frames').glob(f'*'):
        p.unlink()


#############################################

# save settings

import keyframed.serialization
# verbose:
#txt = keyframed.serialization.to_yaml(curved_settings, simplify=True)

# significantly less verbose:
simplified_settings = {}
simplified_settings__curves = {}
for param, curve in curved_settings.parameters.items():
    kf0 = curve._data[0]
    if (len(curve.keyframes) == 1) and (kf0.interpolation_method in ['eased_lerp', 'previous','linear']):
        simplified_settings[param] = kf0.value
    else:
        d_ = curve.to_dict(simplify=True, for_yaml=True)
        d_.pop('label')
        simplified_settings__curves[param] = d_
simplified_settings.update(simplified_settings__curves) # move verbose stuff to the bottom

txt = OmegaConf.to_yaml(OmegaConf.create({'parameters':simplified_settings}))

with open(outdir / 'settings.yaml', 'w') as f:
    f.write(txt)

#########################

# save prompts

prompts_out = []
for prompt_ in prompts:
    rec = {'prompt':prompt_.text}
    curve = prompt_.weight
    # TO DO: YEESH.
    #kf0 = next(curve._data.values())#[0]
    #if (len(curve.keyframes) == 1) and (kf0.interpolation_method in ['eased_lerp', 'previous','linear']):
    #    rec['weight'] = kf0.value
    #else:
    rec['weight'] = curve.to_dict(simplify=True, for_yaml=True)
    # don't reorder prompts. order matters, esp first prompt.
    prompts_out.append(rec)

prompts_yaml = OmegaConf.to_yaml(OmegaConf.create({'prompts':prompts_out}))

with open(outdir / 'prompts.yaml', 'w') as f:
    f.write(prompts_yaml)

##########################


sample_mcmc_klmc2(
    sd_model=sd_model,
    init_image=init_image,
    height=height,
    width=width,
    n=n,
    hvp_method=hvp_method,
    prompts=prompts,
    settings=curved_settings,
    resume=resume,
    resume_from=resume_from,
    img_init_steps=img_init_steps,
    stuff_to_plot=stuff_to_plot,
    checkpoint_every=checkpoint_every,
)


In [ ]:
#@title Make the video

if 'width' not in locals():
    width = height = 512
if 'outdir' not in locals():
    from pathlib import Path
    outdir = Path('frames')

# @markdown If your video is larger than a few MB, attempting to embed it will probably crash
# @markdown the session. If this happens, view the generated video after downloading it first.
embed_video = True # @param {type:'boolean'}
download_video = False # @param {type:'boolean'}

upscale_video = False  # @param {type:'boolean'}


outdir_str = str(outdir)

fps = 14 # @param {type:"integer"}
out_fname = "out.mp4" # @param {type: "string"}

out_fullpath = str( outdir / out_fname )
print(f"Video will be saved to: {out_fullpath}")

compile_video_cmd = f"ffmpeg -y -r {fps} -i 'out_%*.png' -crf 15 -preset veryfast -pix_fmt yuv420p " # {out_fname}"
if upscale_video:
    compile_video_cmd += f"-vf scale={2*width}x{2*height}:flags=lanczos "
compile_video_cmd += f"{out_fname}"

print('\nMaking the video...\n')
!cd {outdir_str}; {compile_video_cmd}


debug=False
if debug:
    #outdir_str = "debug_frames"
    print("\nMaking debug video...")
    #!cd debug_frames; ffmpeg -y -r {fps} -i 'prompts_out_%*.png' -crf 15 -preset veryslow -pix_fmt yuv420p debug_out.mp4
    !cd {debug_dir}; ffmpeg -y -r {fps} -i 'debug_out_%*.png' -crf 15 -preset veryslow -pix_fmt yuv420p debug_out.mp4

if embed_video:
    print('\nThe video:')
    show_video(out_fullpath)
    if debug:
        show_video(debug_dir / "debug_out.mp4")

if download_video and probably_using_colab:
    from google.colab import files
    files.download(out_fullpath)

In [ ]:
#@title Licensed under the MIT License { display-mode: "form" }

# Copyright (c) 2022 Katherine Crowson <crowsonkb@gmail.com>
# Copyright (c) 2023 David Marx <david.marx84@gmail.com>
# Copyright (c) 2022 deforum and contributors

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.